<a href="https://colab.research.google.com/github/R-Damanhuri/ASAG-Indonesia-Matching-Matrix-BiLSTM/blob/main/Finetuning_IndoSBERT_with_ASAG_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pretrained Sentence-Embedding Model Bahasa Indonesia

## IndoSBERT Model (Denaya)

**Tautan**

https://huggingface.co/denaya/indoSBERT-large

**Model Dasar**

IndoBERT (BERT yang dilatih dengan dataset Indonesian Wikipedia (74M words), news articles from Kompas, Tempo (Tala et al., 2003) and Liputan6 (55M words in total), dan Indonesian Web Corpus (Medved and Suchomel, 2017) (90M words)) <br>
Varian: indobert-large-p1

**Fine-tuning**

- Dataset: Translated STS Dataset (Semantic textual similarity)
- Target pada dataset: Score 0-5
- Loss: CosineSimilarityLoss

# Fine-tuning Sentence-Embedding Model

**Sentence-Embedding Model**
1. IndoSBERT (Denaya)

**Dataset Experiments**
1. ASAG Dataset (asag_cleaned) <br>
   This is actual point-based short answer dataset that will be used for ASAG model testing.

In [1]:
!pip install -q datasets sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.3 MB/s eta 0:00:00


In [2]:
!pip install huggingface_hub

## ASAG Dataset

In [3]:
from datasets import load_dataset

asag = load_dataset("damand2061/asag_cleaned")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/170 [00:00<?, ? examples/s]

In [4]:
asag

DatasetDict({
    train: Dataset({
        features: ['Soal', 'Jawaban', 'Nilai_1', 'Nilai_2', 'Rata-rata'],
        num_rows: 679
    })
    validation: Dataset({
        features: ['Soal', 'Jawaban', 'Nilai_1', 'Nilai_2', 'Rata-rata'],
        num_rows: 170
    })
})

In [5]:
from sentence_transformers import InputExample

train_examples = [InputExample(texts=[item['Soal'], item['Jawaban']], label=float(item['Rata-rata'])) for item in asag['train']]
dev_examples = [InputExample(texts=[item['Soal'], item['Jawaban']], label=float(item['Rata-rata'])) for item in asag['validation']]

print(train_examples[:5])
print(dev_examples[:5])

[<sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd4b85e0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd4b8ca0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd4b8d60>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd4b8df0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd4b8e80>]
[<sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd51cc70>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd51c9a0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd51ca00>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd51cd60>, <sentence_transformers.readers.InputExample.InputExample object at 0x7b5acd51ce20>]


In [6]:
from sentence_transformers import SentenceTransformer

model_id = "denaya/indoSBERT-large"

In [7]:
model_asag_cosine = SentenceTransformer(model_id)

**Fine-tuning Model**

Loss function menggunakan CosineSimilarityLoss

In [8]:
import math
from sentence_transformers import losses, evaluation
from torch.utils.data import DataLoader

train_batch_size = 8

#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model_asag_cosine)

# Evaluate the original model on the STS benchmark dataset
dev_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(
    dev_examples,
    show_progress_bar=True,
    batch_size=train_batch_size,
    name='asag-dev'
)

num_epochs = 1
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up
evaluation_steps = int(len(train_dataloader) * 0.1)

model_save_path = 'IndoSAGSBERT'

#Tune the model
model_asag_cosine.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    evaluator=dev_evaluator,
    evaluation_steps=evaluation_steps,
    output_path=model_save_path
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/85 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

**Simpan Model**

Model disimpan ke HuggingFace.

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="IndoSAGSBERT")

RepoUrl('https://huggingface.co/damand2061/IndoSAGSBERT', endpoint='https://huggingface.co', repo_type='model', repo_id='damand2061/IndoSAGSBERT')

In [11]:
files_to_push_to_hub = [
    'README.md',
    'config.json',
    'config_sentence_transformers.json',
    'model.safetensors',
    'modules.json',
    'sentence_bert_config.json',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

for filename in files_to_push_to_hub:
  api.upload_file(
      path_or_fileobj="/content/IndoSAGSBERT/"+filename,
      repo_id="damand2061/IndoSAGSBERT",
      path_in_repo=filename,
      repo_type="model",
      commit_message="Add IndoSAGSBERT model",
      commit_description="Add fine-tuned IndoSBERT on ASAG dataset with cosine similarity loss function"
  )

api.upload_folder(
    folder_path="/content/IndoSAGSBERT/1_Pooling",
    path_in_repo="1_Pooling",
    repo_id="damand2061/IndoSAGSBERT",
    repo_type="model",
)

api.upload_folder(
    folder_path="/content/IndoSAGSBERT/2_Dense",
    path_in_repo="2_Dense",
    repo_id="damand2061/IndoSAGSBERT",
    repo_type="model",
)

api.upload_folder(
    folder_path="/content/IndoSAGSBERT/eval",
    path_in_repo="eval",
    repo_id="damand2061/IndoSAGSBERT",
    repo_type="model",
)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

'https://huggingface.co/damand2061/IndoSAGSBERT/tree/main/eval'